In [ ]:
# ==============================================================================
# CELL 1: IMPORTS AND STYLE CONFIGURATION
# ==============================================================================
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
import os
import scienceplots # Ensure this is installed: pip install scienceplots

# Apply a professional style suitable for IEEE publications
plt.style.use(['science', 'ieee', 'no-latex'])

# This magic command ensures plots are displayed directly in the notebook
%matplotlib inline

print("Libraries imported and style set.")

In [ ]:
# ==============================================================================
# CELL 2: CONFIGURATION AND FILE PATHS
# ==============================================================================

# --- General Configuration ---
OUTPUT_FOLDER = '' # Folder to save the final plot
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# TVLA significance threshold
CRITICAL_VALUE = 4.5

# Data acquisition parameters 
PP_CC_SIM  = 2  # Points per clock cycle in simulation (pre/post edge)
PP_CC_REAL = 16 # Samples per clock cycle in real measurement

# --- Path Definitions for each security level ---
base_sim_path   = '' 
base_real_path  = '' 

# Dictionaries to hold paths and parameters for each version
CONFIGS = {
    '(+)': {
        'sim_fixed': os.path.join(base_sim_path, 'hp/fixed/'),
        'sim_random': os.path.join(base_sim_path, 'hp/random/'),
        'real_fixed': os.path.join(base_real_path, 'TRACES_EDDSA_HP/TRACES_FIXED/'),
        'real_random': os.path.join(base_real_path, 'TRACES_EDDSA_HP/TRACES_RANDOM/'),
        'n_sim_traces': 100,
        'n_real_traces': 100000,
        'cycles': 3145,
        'sim_offset': 1,  # Specify any initial offset to trim from sim traces
        'real_offset': 32 + 2*PP_CC_REAL, # Specify any initial offset to trim from real traces
        'ylim': (-50, 50) # Custom Y-axis limit for this plot
    },
    '(++)': {
        'sim_fixed': os.path.join(base_sim_path, 'hpcm/fixed/'),
        'sim_random': os.path.join(base_sim_path, 'hpcm/random/'),
        'real_fixed': os.path.join(base_real_path, 'TRACES_EDDSA_HPCM/TRACES_FIXED/'),
        'real_random': os.path.join(base_real_path, 'TRACES_EDDSA_HPCM/TRACES_RANDOM/'),
        'n_sim_traces': 2000,
        'n_real_traces': 100000,
        'cycles': 3420, 
        'sim_offset': 1,
        'real_offset': 32 + 2*PP_CC_REAL,
        'ylim': (-30, 30) # Custom Y-axis limit for this plot
    },
    '(+++)': {
        'sim_fixed': os.path.join(base_sim_path, 'hpcm_sm/fixed/'),
        'sim_random': os.path.join(base_sim_path, 'hpcm_sm/random/'),
        'real_fixed': os.path.join(base_real_path, 'TRACES_EDDSA_HPCM_SM/TRACES_FIXED/'),
        'real_random': os.path.join(base_real_path, 'TRACES_EDDSA_HPCM_SM/TRACES_RANDOM/'),
        'n_sim_traces': 2000,
        'n_real_traces': 1000000,
        'cycles': 4620, # Adjust if different
        'sim_offset': 1 + 2*78,
        'real_offset': 32 + 2*PP_CC_REAL,
        'ylim': (-30, 30) # Custom Y-axis limit for this plot
    }
}

print("Configuration and paths are set.")

In [ ]:
# ==============================================================================
# CELL 3: HELPER FUNCTIONS
# ==============================================================================

def load_traces(folder_path, num_traces, dtype):
    """Dynamically loads a set of traces from binary files into a numpy array."""
    try:
        first_trace_path = os.path.join(folder_path, "trace_0.bin")
        num_samples = np.fromfile(first_trace_path, dtype=dtype).shape[0]
    except FileNotFoundError:
        print(f"ERROR: Could not find trace_0.bin in {folder_path} to determine trace size.")
        return None

    trace_data = np.zeros((num_traces, num_samples), dtype=dtype)
    print(f"Loading {num_traces} traces from {folder_path}...")
    for i in range(num_traces):
        trace_file = os.path.join(folder_path, f"trace_{i}.bin")
        if os.path.exists(trace_file):
            trace_data[i, :] = np.fromfile(trace_file, dtype=dtype)
        else:
            print(f"Warning: Trace file not found: {trace_file}")
            
    return trace_data

def calculate_t_score(fixed_traces, random_traces):
    """Performs Welch's t-test and returns the t-statistic scores."""
    t_test_result = ss.ttest_ind(fixed_traces, random_traces, axis=0, equal_var=False)
    t_score = np.nan_to_num(t_test_result[0])
    return t_score

def downsample_real_t_score(t_score_raw, total_cycles, pp_cc_real):
    """Downsamples a high-resolution t-score trace to one point per cycle using max absolute value."""
    downsampled_score = np.zeros(total_cycles)
    if len(t_score_raw) < total_cycles * pp_cc_real:
        print(f"Warning: Real trace length ({len(t_score_raw)}) is shorter than expected ({total_cycles * pp_cc_real}). Adjusting cycle count.")
        total_cycles = len(t_score_raw) // pp_cc_real

    for i in range(total_cycles):
        start_idx = i * pp_cc_real
        end_idx = start_idx + pp_cc_real
        cycle_window = t_score_raw[start_idx:end_idx]
        
        if len(cycle_window) > 0:
            max_abs_idx = np.argmax(np.abs(cycle_window))
            downsampled_score[i] = cycle_window[max_abs_idx]
            
    return downsampled_score

print("Helper functions defined.")

In [ ]:
# CELL 4: DATA PROCESSING AND PLOTTING
# ==============================================================================

# Create a figure with 3 independent subplots
# We set sharex=False and sharey=False to allow for independent scaling
fig, axes = plt.subplots(
    nrows=3, 
    ncols=1, 
    figsize=(12, 7), # Increased height to accommodate independent axes
)

security_levels = ['(+)', '(++)', '(+++)']

# --- Loop through each security level to process and plot ---
for i, level in enumerate(security_levels):
    print(f"\n--- Processing security level: {level} ---")
    
    config = CONFIGS[level]
    ax = axes[i] # Select the current subplot

    # --- 1. Load and Process Simulated Data ---
    sim_fixed_data = load_traces(config['sim_fixed'], config['n_sim_traces'], np.int32)
    sim_random_data = load_traces(config['sim_random'], config['n_sim_traces'], np.int32)
    
    sim_fixed_data = sim_fixed_data[:, config['sim_offset']:]
    sim_random_data = sim_random_data[:, config['sim_offset']:]

    sim_t_score_raw = calculate_t_score(sim_fixed_data, sim_random_data)
    # Extract posedge sample
    sim_t_score = sim_t_score_raw[::PP_CC_SIM] 

    # --- 2. Load and Process Real Data ---
    real_fixed_data = load_traces(config['real_fixed'], config['n_real_traces'], np.float32)
    real_random_data = load_traces(config['real_random'], config['n_real_traces'], np.float32)

    real_fixed_data = real_fixed_data[:, config['real_offset']:]
    real_random_data = real_random_data[:, config['real_offset']:]
    
    real_t_score_raw = calculate_t_score(real_fixed_data, real_random_data)
    real_t_score = downsample_real_t_score(real_t_score_raw, config['cycles'], PP_CC_REAL)

    # --- 3. Plotting for the current security level ---
    min_len = min(len(sim_t_score), len(real_t_score))
    x_axis = np.arange(min_len)

    
    ax.plot(x_axis, real_t_score[:min_len], color='blue', linewidth=0.6, label='Real t-test')
    ax.plot(x_axis, sim_t_score[:min_len], color='green', linestyle='--', linewidth=0.5, alpha=0.8, label='Simulated t-test')
    
    # Add TVLA threshold lines
    ax.axhline(y=CRITICAL_VALUE, color='red', linestyle='--', linewidth=1, label='Threshold')
    ax.axhline(y=-CRITICAL_VALUE, color='red', linestyle='--', linewidth=1)
    
    # Customize subplot
    ax.set_title(f'TVLA of EdDSA25519 Signature Generation {level}', fontsize=14)
    ax.set_ylabel('t-statistic', fontsize=12)
    ax.set_xlim(0, config['cycles']) # Use the specific cycle count for this plot's x-axis
    ax.set_ylim(config['ylim']) # Use the specific y-limit for this plot

# --- 4. Final Figure Customization ---
axes[-1].set_xlabel('Clock Cycles', fontsize=12, fontweight='bold')

# Create a single legend for the entire figure, placed neatly at the top
handles, labels = axes[0].get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))
axes[0].legend(unique_labels.values(), unique_labels.keys(), 
               loc='lower center',          # The part of the legend box to anchor
               bbox_to_anchor=(0.5, 1.15),  # Position: 50% across the axis, 15% above the top of the axis
               ncol=3,                      # Number of columns for a horizontal layout
               fontsize=14,
               frameon=True,
               fancybox=True)

# Use standard tight_layout. It will automatically adjust spacing for everything,
# including the legend we placed just above the top subplot.
plt.tight_layout()

# Save the combined figure
output_path = os.path.join(OUTPUT_FOLDER, 'tvla_comparison_all_levels.pdf')
plt.savefig(output_path, dpi=300)

print(f"\nCombined TVLA plot saved to {output_path}")
plt.show()